## 퀴즈
### abalone file에 대해 70%로 학습하고 30% 대해 결과를 구하세요

In [17]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
import numpy
from sklearn import preprocessing, model_selection



In [22]:
def read_abalone():
    abalone = pd.read_csv('data/abalone.data', header=None)
    # print(abalone)

    # 엔코딩
    enc = preprocessing.LabelEncoder()
    y = enc.fit_transform(abalone[0].values)
    print(y)

    x = abalone.values[:, 1:]  # 행, 열
    x = np.float32(x)
    print(x.dtype)

    return x, y
read_abalone()

[2 2 0 ... 2 0 2]
float32


(array([[ 0.455 ,  0.365 ,  0.095 , ...,  0.101 ,  0.15  , 15.    ],
        [ 0.35  ,  0.265 ,  0.09  , ...,  0.0485,  0.07  ,  7.    ],
        [ 0.53  ,  0.42  ,  0.135 , ...,  0.1415,  0.21  ,  9.    ],
        ...,
        [ 0.6   ,  0.475 ,  0.205 , ...,  0.2875,  0.308 ,  9.    ],
        [ 0.625 ,  0.485 ,  0.15  , ...,  0.261 ,  0.296 , 10.    ],
        [ 0.71  ,  0.555 ,  0.195 , ...,  0.3765,  0.495 , 12.    ]],
       dtype=float32),
 array([2, 2, 0, ..., 2, 0, 2]))

In [25]:
def model_1(x_train, y_train, x_test):
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(0.0001),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics='acc')

    model.fit(x_train, y_train, epochs=50, verbose=2)
    return model.predict(x_test, verbose=0)

In [30]:
def model_2(x_train, y_train, x_test, y_test, n0, n1, n2):
    model = keras.Sequential([
        keras.layers.Dense(n0, activation='relu'),
        keras.layers.Dense(n1, activation='relu'),
        keras.layers.Dense(n2, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(0.0001),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics='acc')

    model.fit(x_train, y_train, epochs=50, verbose=0)
    print(model.evaluate(x_test, y_test, verbose=0))
    return model.predict(x_test, verbose=0)

model_2()

TypeError: model_2() missing 7 required positional arguments: 'x_train', 'y_train', 'x_test', 'y_test', 'n0', 'n1', and 'n2'

### 앙상블 모델 사용

In [28]:
x, y = read_abalone()

x = preprocessing.scale(x)
# x = preprocessing.minmax_scale(x)

data = model_selection.train_test_split(x, y, train_size=0.7)
x_train, x_test, y_train, y_test = data

ensemble = np.zeros([len(y_test), 3])
for layers in [(128, 64, 16), (64, 32, 12), (64, 16, 8),
               (128,48, 16), (64, 16, 16), (128, 32, 32), (256, 64, 64)]:
    p = model_2(x_train, y_train, x_test, y_test, *layers)
    # print(p.shape)                        # (1254, 3)

    ensemble += p

p_arg = np.argmax(ensemble, axis=1)
print('acc :', np.mean(p_arg == y_test))

[2 2 0 ... 2 0 2]
float32


C:\Users\Harmony05\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\Harmony05\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


[0.8223028182983398, 0.5789473652839661]
[0.8294222950935364, 0.5590111613273621]
[0.8298503160476685, 0.5629984140396118]
[0.8240167498588562, 0.5869218707084656]
[0.8294695019721985, 0.5789473652839661]
[0.8228338956832886, 0.5837320685386658]
[0.8192406296730042, 0.5837320685386658]
acc : 0.5829346092503987
